# Load Data

In [ ]:
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
import os
import matplotlib.pyplot as plt
import math
import numpy as np
sd = datetime(1980,1,1)
ed = datetime.today()
df= pd.DataFrame(index=pd.date_range(start=sd,end=ed, freq='D'))
#df = pd.DataFrame()
investments ={
   "Bitcoin":"BTC-USD",    
    "Tesla":"TSLA",    
    "S&P500":"VFINX",
    "Bond":"VBMFX",
    "Gold": "GC=F",
}

investments_data = "investments_data.csv"
s=[]
if (os.path.exists(investments_data)):
    print("Load existing data")
    df = pd.read_csv(investments_data, parse_dates=True,index_col='Date')
else:
    for name in investments.keys():
        s = (pdr.get_data_yahoo(symbols=investments[name], start=sd, end=ed)["Adj Close"])
        #drop duplicated index
        s = s.groupby(level=0).last()
        df[name]=s
    df.to_csv(path_or_buf=investments_data,index_label='Date')
print(df.head(10),)
print(df.tail(10),)
#df.columns

In [ ]:
sd = datetime(2015,1,1)
ed = datetime(2019,12,31)
truncated_df = df.fillna(method='ffill').truncate(before=sd,after=ed)

print(truncated_df.head(10))
print(truncated_df.tail(10))

In [ ]:
from lib import drawdown,plot_drawdown,cagr

# Get MDD
print("MDD")
for name in investments.keys():
    print(drawdown(truncated_df[name]).min())
print("CAGR")
# Get CAGR
period=720
for name in investments.keys():
    start =(truncated_df[name].iloc[0])
    end = (truncated_df[name].iloc[-1])
    cagrs = truncated_df[name].rolling(period).apply(func=lambda v:cagr(v.iloc[0],v.iloc[-1],period),raw=False)
    total_cagr = cagr(start,end, 365*4)
    print(cagrs.std())

In [ ]:

drawdown_period =30

for name in investments.keys():
    print(name)
    s=df[name].fillna(method='ffill')
    drawdowns = s.rolling(drawdown_period).apply(func=drawdown,raw=False)
    mdd = drawdowns.min()
    x=list(map(lambda v:drawdowns.quantile(math.exp(-v*0.1)),np.arange(0,50)))
    print(drawdowns.quantile(0.01))
    y = list(map(lambda v:math.log((drawdowns<=v).sum()/drawdowns.count()),x))
    plt.scatter (x,y,label=name)
    plt.legend()
plt.title(f'Drawdown in {period} days')
plt.xlabel('Drawdown')
plt.ylabel('log(P(x))')

In [ ]:
check_period=720
q_values=(0.1,0.03,0.01)
period=720
drawdown_periods=(10,30,90,720)
for name in investments.keys():
    print(name)
    s=df[name].fillna(method='ffill')
    print(s.count())
    for drawdown_period in drawdown_periods:
        print (f'Drawdown period {drawdown_period}')
        drawdowns = s.rolling(drawdown_period).apply(func=drawdown,raw=False)
        print(f'xxx: {drawdowns.count()}')
        for q in q_values:
            print(f'Q:{q}')
            quantiles = drawdowns.rolling(check_period).apply(lambda v:v.quantile(q),raw=False)
            avg = quantiles.mean()
            std = quantiles.std()
            print(f'avg: {avg}, std: {std}, releatvie std:{std/avg}')